<h1 style="text-decoration :underline;color:red">Création d'un dataset propre pour l'algo</h1>

<p style="color:blue">Le but de ce notebook est de créer un dataset prêt pour l'algo de prédiction du vainqueur. Pour cela, nous allons le nettoyer, ajouter des features</p>

<h2 style="text-decoration :underline;color:green">Module utilisé lors de ce notebook :</h2>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

from datetime import datetime, timedelta 
from sklearn.model_selection import train_test_split

pd.set_option('max_columns', 50)


<h2 style="text-decoration :underline;color:green">lecture du fichier, nettoyage, transformation de donnée :</h2>

In [3]:
df = pd.read_csv("atp_data.csv")

In [4]:
#conversion en datatime
df.Date = pd.to_datetime(df.Date)
date = pd.DataFrame(df.Date)

In [5]:
df.tail()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,Wsets,Lsets,Comment,PSW,PSL,B365W,B365L,elo_winner,elo_loser,proba_elo
44703,17,Dubai,Dubai Tennis Championships,2018-03-03,ATP500,Outdoor,Hard,The Final,3,Bautista Agut R.,Pouille L.,23,15,2.0,0.0,Completed,1.65,2.43,1.66,2.20,1835.901543,1842.459807,0.490563
44704,16,Acapulco,Abierto Mexicano,2018-03-03,ATP500,Outdoor,Hard,Semifinals,3,Anderson K.,Donaldson J.,8,59,2.0,1.0,Completed,1.34,3.63,1.30,3.50,1843.576914,1647.967561,0.755103
44705,18,Sao Paulo,Brasil Open,2018-03-03,ATP250,Indoor,Clay,Semifinals,3,Jarry N.,Zeballos H.,73,69,2.0,1.0,Completed,1.85,2.09,1.90,1.90,1588.293069,1619.758421,0.454841
44706,16,Acapulco,Abierto Mexicano,2018-03-04,ATP500,Outdoor,Hard,The Final,3,Del Potro J.M.,Anderson K.,9,8,2.0,0.0,Completed,1.52,2.75,1.50,2.62,1966.532849,1851.413609,0.659862
44707,18,Sao Paulo,Brasil Open,2018-03-04,ATP250,Indoor,Clay,The Final,3,Fognini F.,Jarry N.,20,73,2.0,1.0,Completed,1.38,3.38,1.36,3.25,1814.065865,1605.738153,0.768388


In [6]:
len(df)

44708

In [5]:
#on enlève tout les matchs qui ont été gagnés irrégulièrement
match_non_fini = df[(df.Comment == 'Retired')| (df.Comment == "Walkover") | (df.Comment == "Disqualified")].index
df = df.drop(match_non_fini)
df.index = range(len(df.index))

<h2 style="text-decoration :underline;color:green">Réagencement des variables en fonction de Favori et Challenger (non winner / loser) :</h2>

<p style="color:blue"> Il est nécessaire de changer comment sont rangées nos données car la variable WRank donne 2 infos, le rang du joueur mais aussi le fait que ce joueur est le vainqueur. par conséquent, rentrer un joueur dans la variable WRank c'est le placer comme vainqueur alors qu'on ne le sait pas encore. nous allons donc créer 2 variables Favori et Challenger basé sur le rang ATP des 2 joueurs afin de ne pas rentrer d'informations biaisées</p>

In [9]:
#On crée donc la variable vainqueur qui aura pour valeur soit favori soit challenger
df["Vainqueur"]= "wait"

for i in range(len(df)):
    if (df["WRank"][i] <= df["LRank"][i]):
        df["Vainqueur"][i] = "favori"
    else :
        df["Vainqueur"][i] = "challenger"

<ipython-input-9-34db60acc3fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vainqueur"][i] = "favori"
<ipython-input-9-34db60acc3fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vainqueur"][i] = "challenger"


In [7]:
#on met ces données de côtés pour les concatener par la suite.
info_match = df[["ATP","Location","Tournament","Date","Series","Court","Surface","Round","Best of","Comment","Wsets","Lsets","Vainqueur","Winner"]]

In [9]:
#créons un dataframe avec un favori( meilleur rang mondial) et un challenger (moins bon rang mondial) et les stats des joueurs qui en découlent.
data = pd.DataFrame(columns=["Favori","Rank_F","PS_F","B365_F","elo_F","proba_elo_F",'TV_F', 'TV_surf_F','sets_F',
                               "Challenger","Rank_C","PS_C","B365_C","elo_C","proba_elo_C",'TV_C',,'TV_surf_C','sets_C'])

for i in range(len(df)):
        match = df.iloc[i,:]
        #le Winner est le favori
        if match.WRank < match.LRank:
            data.loc[i,"Favori"] = match.Winner
            data.loc[i,"Rank_F"] = match.WRank
            data.loc[i,"PS_F"] = match.PSW
            data.loc[i,"B365_F"] = match.B365W
            data.loc[i,"elo_F"] = match.elo_winner
            data.loc[i,"proba_elo_F"] = match.proba_elo
            
            data.loc[i,"Challenger"] = match.Loser
            data.loc[i,"Rank_C"] = match.LRank
            data.loc[i,"PS_C"] = match.PSL
            data.loc[i,"B365_C"] = match.B365L
            data.loc[i,"elo_C"] = match.elo_loser
            data.loc[i,"proba_elo_C"] = 1 - match.proba_elo
        #le Winner est le challenger
        else:
            data.loc[i,"Favori"] = match.Loser
            data.loc[i,"Rank_F"] = match.LRank
            data.loc[i,"PS_F"] = match.PSL
            data.loc[i,"B365_F"] = match.B365L
            data.loc[i,"elo_F"] = match.elo_loser
            data.loc[i,"proba_elo_F"] = 1 - match.proba_elo
            
            data.loc[i,"Challenger"] = match.Winner
            data.loc[i,"Rank_C"] = match.WRank
            data.loc[i,"PS_C"] = match.PSW
            data.loc[i,"B365_C"] = match.B365W
            data.loc[i,"elo_C"] = match.elo_winner
            data.loc[i,"proba_elo_C"] = match.proba_elo

        

In [10]:
#maintenant qu'on a réagencé les données liées auxjoueurs on rajoute les données du match.
data = data.join(info_match)

<h2 style="text-decoration :underline;color:green">Ajout de features pour chaque joueur basé sur les 150 derniers jours d'avant match :</h2>

<p style="color:blue">La cellule suivante va créer des features basées sur les performances du joueur sur les 150 derniers jours précédent son match. Cela permet d'évaluer l'état de forme du joueur et d'ajouter des données pouvant améliorer la prédiction du vainqueur.</p>

In [12]:
#fonction qui permet de récupérer les matchs effectués par le joueur A 150 jours avant son match i
def match_150_avant(i):
    #on obtient le match qui se prépare
    match = data.iloc[i,:]
    #on garde les matchs précédents uniquement.
    match_avant = data.iloc[:i-1,:]
    joueurA = match_avant[(match_avant["Favori"] == match.Favori) | (match_avant["Challenger"] == match.Favori)]
    joueurB = match_avant[(match_avant["Favori"] == match.Challenger) | (match_avant["Challenger"]== match.Challenger)]
    joueurA = joueurA[(joueurA.Date <= match.Date) & (joueurA.Date >= (match.Date - timedelta(days=150)))]
    joueurB = joueurB[(joueurB.Date <= match.Date) & (joueurB.Date >= (match.Date - timedelta(days=150)))]
    joueurA = joueurA.reset_index().drop("index",axis=1)
    joueurB = joueurB.reset_index().drop("index",axis=1)
    
    #ajouter le taux de victoire des joueurs sur les 150 derniers jours
    if (len(joueurA)== 0):
        data.loc[i,"TV_F"] = 0
    else:
        data.loc[i,"TV_F"] = len(joueurA[joueurA["Winner"]== match.Favori])/len(joueurA)
    if (len(joueurB)== 0):    
        data.loc[i,"TV_C"] = 0
    else:
        data.loc[i,"TV_C"] = len(joueurB[joueurB["Winner"]== match.Challenger])/len(joueurB)
    
    
    #progression par rapport à sa moyenne des elo points sur cette période
    data.loc[i,"moy_elo_F"]=  match.elo_F - joueurA.elo_F.mean()
    data.loc[i,"moy_elo_C"]=  match.elo_C - joueurB.elo_C.mean()
    
    
    #ajouter le taux de victoire sur la surface en question
    surface_A = joueurA[joueurA["Surface"]== match.Surface]
    surface_B = joueurB[joueurB["Surface"]== match.Surface]
    if (len(surface_A)== 0):
        data.loc[i,"TV_surf_F"] = 0
    else:
        data.loc[i,"TV_surf_F"] = len(surface_A[surface_A["Winner"]== match.Favori])/len(surface_A)
    if (len(surface_B)== 0):    
        data.loc[i,"TV_surf_C"] = 0
    else:
        data.loc[i,"TV_surf_C"] = len(surface_B[surface_B["Winner"]== match.Challenger])/len(surface_B)
    
    #nombre de sets joués sur le tournoi en question
    set_A = joueurA[joueurA["Tournament"]== match.Tournament]
    set_B = joueurB[joueurB["Tournament"]== match.Tournament]
    data.loc[i,"sets_F"] = set_A.Wsets.sum()+set_A.Lsets.sum()
    data.loc[i,"sets_C"] = set_B.Wsets.sum()+set_B.Lsets.sum()
    

In [13]:
#on lance la fonction sur le dataframe
for i in range (len(data)):
    match_150_avant(i)

<h2 style="text-decoration :underline;color:green">Préparation à l'export du dataset :</h2>

In [25]:
#on enlève tout les valeurs absentes et les matchs n'ayant pas de cote bookmaker également.
data_clean_all = data_clean_all.dropna()
data_clean_all.to_csv("data_clean_all.csv")

In [26]:
data_clean_all

,Favori,Rank_F,PS_F,B365_F,elo_F,proba_elo_F,TV_F,TV_surf_F,sets_F,Challenger,Rank_C,PS_C,B365_C,elo_C,proba_elo_C,TV_C,TV_surf_C,sets_C,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Comment,Wsets,Lsets,Vainqueur,Winner,moy_elo_F,moy_elo_C
8365,Kiefer N.,58,1.719,1.615,1618.18,0.681306,0.5,0.5,0,Ulihrach B.,246,2.21,2.2,1486.2,0.318694,0.428571,0.5,0,3,Doha,Qatar Exxon Mobil Open,2004-01-05,International,Outdoor,Hard,1st Round,3,Completed,2.0,0.0,challenger,Ulihrach B.,-36.522332,-20.871951
8366,Schuettler R.,6,1.385,1.364,1820.76,0.765687,0.763158,0.72,0,Youzhny M.,43,3.25,2.875,1615.06,0.234313,0.461538,0.428571,0,3,Doha,Qatar Exxon Mobil Open,2004-01-05,International,Outdoor,Hard,1st Round,3,Completed,2.0,0.0,challenger,Youzhny M.,19.791315,41.588209
8367,Sargsian S.,39,1.746,1.571,1654.47,0.594046,0.666667,0.6,0,Carraz G.,67,2.18,2.25,1588.33,0.405954,0.571429,0.555556,0,3,Doha,Qatar Exxon Mobil Open,2004-01-05,International,Outdoor,Hard,1st Round,3,Completed,2.0,1.0,favori,Sargsian S.,15.865896,40.962208
8371,Arazi H.,52,2.06,2.1,1584.48,0.522221,0.571429,0.5,0,Enqvist T.,96,1.82,1.667,1569.03,0.477779,0.461538,0.454545,0,3,Doha,Qatar Exxon Mobil Open,2004-01-05,International,Outdoor,Hard,1st Round,3,Completed,2.0,1.0,favori,Arazi H.,-4.357656,34.077361
8372,van Lottum J.,92,1.455,1.444,1452.26,0.440959,0.444444,0.333333,0,Blanco G.,99,2.9,2.625,1493.48,0.559041,0.666667,0,0,2,Chennai,TATA Open,2004-01-05,International,Outdoor,Hard,1st Round,3,Completed,2.0,0.0,favori,van Lottum J.,-61.224824,-10.241221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43010,Pouille L.,15,2.43,2.2,1842.46,0.509437,0.772727,0.772727,11,Bautista Agut R.,23,1.65,1.66,1835.9,0.490563,0.631579,0.631579,9,17,Dubai,Dubai Tennis Championships,2018-03-03,ATP500,Outdoor,Hard,The Final,3,Completed,2.0,0.0,challenger,Bautista Agut R.,48.408738,165.883261
43011,Anderson K.,8,1.34,1.3,1843.58,0.755103,0.611111,0.611111,6,Donaldson J.,59,3.63,3.5,1647.97,0.244897,0.428571,0.428571,4,16,Acapulco,Abierto Mexicano,2018-03-03,ATP500,Outdoor,Hard,Semifinals,3,Completed,2.0,1.0,favori,Anderson K.,29.784931,46.204092
43012,Zeballos H.,69,2.09,1.9,1619.76,0.545159,0.5,0.571429,9,Jarry N.,73,1.85,1.9,1588.29,0.454841,0.692308,0.8,9,18,Sao Paulo,Brasil Open,2018-03-03,ATP250,Indoor,Clay,Semifinals,3,Completed,2.0,1.0,challenger,Jarry N.,-61.940388,79.942472
43013,Anderson K.,8,2.75,2.62,1851.41,0.340138,0.666667,0.666667,9,Del Potro J.M.,9,1.52,1.5,1966.53,0.659862,0.8,0.8,9,16,Acapulco,Abierto Mexicano,2018-03-04,ATP500,Outdoor,Hard,The Final,3,Completed,2.0,0.0,challenger,Del Potro J.M.,39.699484,220.827893


Nous possédons à présent un dataset qui nous informe sur les caractéristiques et l'état de forme de des deux joueurs avant leur match sans pour autant connaître le vainqueur.